In [2]:
import pandas as pd
from progressbar import ProgressBar
import csv
from time import time  # To time our operations
import os
os.listdir()
print(os.listdir("Data/"))#Contents of the Data folder

['cz_item_category', 'item_content_cz', 'item_content_viewed_sid.csv', 'item_cz', 'list_category', 'material_attributes_cz', 'product_views_cz', 'product_views_cz_filtered.csv']


In [115]:
product_views_cz = pd.read_csv("Data/product_views_cz_filtered.csv",low_memory=False,encoding="utf-8")
product_views_cz.head()

,user_id_anon,product_id
0,14723304,100011803451
1,14723304,100011612041
2,14723304,100016624290
3,14723304,1093107
4,14723304,100016287573


In [116]:
product_views_cz = product_views_cz.loc[product_views_cz["product_id"]!="ghostery", ["user_id_anon","product_id"]]

In [117]:
print(len(product_views_cz))
product_views_cz["product_id"] = pd.to_numeric(product_views_cz["product_id"]).astype('Int64') #přetypování sloupce product_id
print(product_views_cz.dtypes)

31502086
user_id_anon    int64
product_id      Int64
dtype: object


In [5]:
item_cz = pd.read_csv("Data/item_cz",usecols = ["id","sid"]) #filtrování tabulky item_cz přes produkty, na které se někdo podíval
print(item_cz.head())
print(len(item_cz))

           id           sid
0  1179082107  100022742477
1  1218415675       2530760
2  1184542732  100022850488
3  1179295602  100022753680
4  1179087501  100022749351
10077099


In [7]:
item_cz_viewed = item_cz[item_cz["sid"].isin(set(product_views_cz["product_id"]))]

In [8]:
print(item_cz_viewed)

                  id           sid
1         1218415675       2530760
17           6971559       1346423
61           7829808       1026107
90        1549538267  100033492178
101       1214119508  100023319232
...              ...           ...
10077035     8388216    1259300009
10077082  1120068001  100022270996
10077087     6854287  100002788131
10077094  1497678877  100031278416
10077096  1411098824  100029808946

[1248482 rows x 2 columns]


In [18]:
item_content_cz = pd.read_csv("Data/item_content_cz", usecols=["item_id","name","name_grouping","brief_plain"],nrows=1000000)
item_content_cz.head()

,item_id,name,name_grouping,brief_plain
0,108205131,Silikonové pouzdro iSaprio - Birds - Huawei Y5...,Villager Silikonové pouzdro iSaprio - Birds - ...,Stylový ochranný kryt na mobil je vyrobený z p...
1,9949671,"Interiérové dveře PORTO kombinované, model 2, ...",VASCO DOORS Interiérové dveře PORTO kombinovan...,"Sklo SATINATO, 80 cm, pravé dveře, bez otvoru,..."
2,9766441,Smashed Lemon Dámské šaty Green/Blue 18516,Dámské šaty Green/Blue 18516 (Velikost M),Šaty od známé značky Smashed Lemon. Elegantní ...
3,130824063,pánské jeansy Finsbury PM200338WD2 33/34 tmavě...,Pepe Jeans pánské jeansy Finsbury PM200338WD2,V pánských jeansech Finsbury od značky Pepe Je...
4,830595568,dámská mikina Logo Hoodie 1895751010 L černá,Columbia dámská mikina Logo Hoodie 1895751010,Dámská mikina Columbia Logo Hoodie 1895751010 ...


In [19]:
item_content_viewed = item_content_cz[item_content_cz["item_id"].isin(set(item_cz_viewed["id"]))] #filtrování tabulky item_content_cz přes produkty, na které se někdo podíval
print(len(item_content_viewed))

135843


In [20]:
t = time() #postupné filtrování tabulky item_content_cz přes produkty, na které se někdo podíval

column_names = ["id", "item_id", "name", "name_grouping", "brief_plain"]
starting_row = 1000000
number_rows = 1000000
while starting_row < 10000000:
    print("skipping rows:" + str(starting_row))
    item_content_cz = pd.read_csv("Data/item_content_cz",names = column_names, skiprows = starting_row, nrows = number_rows)
    temp = item_content_cz[item_content_cz["item_id"].isin(set(item_cz_viewed["id"]))]
#     print(len(temp))
    item_content_viewed = pd.concat([item_content_viewed,temp], ignore_index=True)
#     print(len(item_content_viewed))
#     print(item_content_viewed)
    starting_row = starting_row + number_rows
    print('Time to filter: {} mins '.format(round((time() - t) / 60, 2)))

skipping rows:1000000
139891
275734
           item_id                                               name  \
0        130824063  pánské jeansy Finsbury PM200338WD2 33/34 tmavě...   
1        830595568       dámská mikina Logo Hoodie 1895751010 L černá   
2        830129415                dámské šaty 41.003.82.2525 36 modrá   
3          7748770                   Zen Pro Wmn Taupe/Coral Red 41,5   
4        597380208  Plastový kryt - Best Friends pro Samsung Galax...   
...            ...                                                ...   
275729   223205622     HEITRONIC závěsná lampa CARACAS E27 230V 27021   
275730   939148774  Kusový koberec Eton Exklusive žlutý kruh 200x2...   
275731  1447471499  růžové dámské vzorované tenisky s koženými det...   
275732  1582874873                                   Malovaná písanka   
275733  1582842237                                     Mluvíme o sexu   

                                            name_grouping  \
0           Pepe Jeans pán

In [21]:
item_content_viewed = item_content_viewed[["item_id","name","name_grouping","brief_plain"]]
item_content_viewed.head()

,item_id,name,name_grouping,brief_plain
0,130824063,pánské jeansy Finsbury PM200338WD2 33/34 tmavě...,Pepe Jeans pánské jeansy Finsbury PM200338WD2,V pánských jeansech Finsbury od značky Pepe Je...
1,830595568,dámská mikina Logo Hoodie 1895751010 L černá,Columbia dámská mikina Logo Hoodie 1895751010,Dámská mikina Columbia Logo Hoodie 1895751010 ...
2,830129415,dámské šaty 41.003.82.2525 36 modrá,Q/S designed by dámské šaty 41.003.82.2525,Dámské šifonové šaty s vypasovanou plisovanou ...
3,7748770,"Zen Pro Wmn Taupe/Coral Red 41,5",Scarpa Zen Pro WMN taupe/coral red,Scarpa Zen Pro W jsou dámské boty s propracova...
4,597380208,Plastový kryt - Best Friends pro Samsung Galax...,iSaprio Plastový kryt s motivem Best Friends,Plastový kryt je určený pro telefon Samsung Ga...


In [22]:
item_cz_viewed = item_cz_viewed.rename(columns={'id': 'item_id'})
print(item_cz_viewed.head())

        item_id           sid
1    1218415675       2530760
17      6971559       1346423
61      7829808       1026107
90   1549538267  100033492178
101  1214119508  100023319232


In [33]:
item_content_v_id = item_content_viewed.merge(item_cz_viewed, how='left', on='item_id') #spojení tabulek, přidání sloupce sid
print(item_content_v_id.head())

     item_id                                               name  \
0  130824063  pánské jeansy Finsbury PM200338WD2 33/34 tmavě...   
1  830595568       dámská mikina Logo Hoodie 1895751010 L černá   
2  830129415                dámské šaty 41.003.82.2525 36 modrá   
3    7748770                   Zen Pro Wmn Taupe/Coral Red 41,5   
4  597380208  Plastový kryt - Best Friends pro Samsung Galax...   

                                   name_grouping  \
0  Pepe Jeans pánské jeansy Finsbury PM200338WD2   
1  Columbia dámská mikina Logo Hoodie 1895751010   
2     Q/S designed by dámské šaty 41.003.82.2525   
3             Scarpa Zen Pro WMN taupe/coral red   
4   iSaprio Plastový kryt s motivem Best Friends   

                                         brief_plain           sid  
0  V pánských jeansech Finsbury od značky Pepe Je...    1391559011  
1  Dámská mikina Columbia Logo Hoodie 1895751010 ...    1511421004  
2  Dámské šifonové šaty s vypasovanou plisovanou ...    1511205002  
3  Scarp

In [34]:
item_content_v_id = item_content_v_id[["sid","name","name_grouping","brief_plain"]]

In [35]:
# item_content_v_id.to_csv(path_or_buf="Data/item_content_viewed_sid.csv",index=False)

In [94]:
#celkem 1 294 542 řádků
names = ["sid", "name", "name_grouping","brief_plain"]
item_content_v_id = pd.read_csv("Data/item_content_viewed_sid.csv",names = names, skiprows = 1250000, nrows=50000)
print(item_content_v_id)

                sid                                               name  \
0      100020294080                           HD2509R fén Compact Line   
1      100020898450  Dámské tenisky White 5-5-23637-24-100 (Velikos...   
2      100017038506  Dámské tenisky 1-1-23731-24-627 Saffron (Velik...   
3      100016838945                      Narozeninová prskavka číslo 8   
4      100029519706     Sáčky do vysavače ROWENTA Bully 3 in 1 - 12 ks   
...             ...                                                ...   
44538  100004145040        C&K Vocal: Dřív než něco začne (4x CD) - CD   
44539       1010553                                     Sestava Parana   
44540    1416452005                  Classic Dusty Green M8W10 (41-42)   
44541  100003994287  Cantini Barbara: Mortina - Příběh, při kterém ...   
44542  100020709700  LED stolní lampička 2,5W/3000K/170Lm, černá s ...   

                                           name_grouping  \
0                       Grundig HD2509R fén Compact

In [95]:
item_content_v_id_backup = item_content_v_id 

In [96]:
item_content_v_id = item_content_v_id_backup

In [97]:
#All characters to lowercase
#deletes characters: "&" "," "." "(" ")" "/" or two spaces
name_column = "name_grouping"
item_content_v_id.loc[:,name_column] = item_content_v_id[name_column].str.casefold().str.replace('&','').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('.','').str.replace('/',' ').str.replace('  ',' ') 
# material_attributes_cz_no_duplicates.loc[:,"NAME_GROUPING"] = material_attributes_cz_no_duplicates.NAME_GROUPING.str.casefold().str.replace('&','').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('.','').str.replace('/',' ').str.replace('  ',' ') 

In [98]:
t = time()
pbar = ProgressBar()

def removeQuantityColor(setOfNames, name_column): #filtr of properties of product - quantity and color
    colorsCZ = ["červen", "čern", "bíl", "žlut", "zelen", "modr", "růžov", "hněd", "béžov", "tyrkysov", "oranžov", "šed", "celooranžov", "barv", "zlat", "střibrn", " vel "]
    colorsEN = ["red", "black", "white", "yellow", "green", "blue", "pink", "brown", "beige", "turquoise", "orange", "grey", "gold", "silver", "navy"]
    vel = "velikost"
    for j in pbar(setOfNames[name_column]): 
        i = j
        for String in ["ks", "mm", "mah", "cm", "let", "ml", "w", "gb", "mb", "oz", "l", "m", "a", "kg", "g", "lb", "t", "ghz", "hz", "tbl", "měs"]:
            for string in [" " + String, String]:
                index = i.find(string) #find first occurence of string in i
                while index != -1:
                    if (index != 0 and ord(i[index-1]) >= 48 and ord(i[index-1]) < 58 and #check if previous char is a number
                        ((index + len(string) < len(i) and i[index + len(string)] == ' ') or
                          index + len(string) == len(i))):
#                         print("REMOVE " + string)
#                         print(i)
                        i = i[:index]+i[index+len(string):] #slices found string from i
                        while index > 0 and ((ord(i[index-1]) >= 48 and ord(i[index-1]) < 58) or #deleting every number before
                                i[index-1] == "-" or #or characters: -,x,+,times or variants with spaces (before or after)
                                i[index-1] == "x" or
                                i[index-1] == '+' or
                                ord(i[index-1]) == 158 or
                                (index >1 and (i[index-2:index] == 'x ' or #one space before character of after
                                                i[index-2:index] == ' x' or
                                                i[index-2:index] == '- ' or 
                                                i[index-2:index] == ' -' or
                                                i[index-2:index] == '+ ' or 
                                                i[index-2:index] == ' +' or
                                                (i[index-1] == ' ' and ord(i[index-2]) == 158) or
                                                (i[index-2] == ' ' and ord(i[index-1]) == 158) or
                                                (index > 2 and (i[index-3:index] == ' x ' or #spaces on both sides of character
                                                                i[index-3:index] == ' - ' or
                                                                i[index-3:index] == ' + ' or 
                                                               (i[index-1] == ' ' and ord(i[index-2]) == 158 and i[index-3] == ' ')))))):
                            if index > 2 and (i[index-3:index] == ' x ' or #slicing of i - spaces on both sides of character
                                              i[index-3:index] == ' - ' or
                                              i[index-3:index] == ' + ' or
                                             (i[index-1] == ' ' and ord(i[index-2]) == 158 and i[index-3] == ' ')):
                                i = i[:index-3] + i[index:]
                                index = index-3
                            elif index > 2 and (ord(i[index-3]) >= 48 and ord(i[index-3]) < 58) and \
                                             (i[index-2:index] == 'x ' or #slicing of i - space on 1 side of character
                                              i[index-2:index] == ' x' or
                                              i[index-2:index] == '- ' or
                                              i[index-2:index] == ' -' or
                                              i[index-2:index] == '+ ' or
                                              i[index-2:index] == ' +' or
                                             (i[index-1] == ' ' and ord(i[index-2]) == 158) or 
                                             (i[index-2] == ' ' and ord(i[index-1]) == 158)):
                                i = i[:index-2] + i[index:]
                                index = index-2
                            elif index == 1 or (index > 1 and #slicing one character from i
                                              (i[index-2:index] != 'x ' and
                                               i[index-2:index] != ' x' and
                                               i[index-2:index] != '- ' and
                                               i[index-2:index] != ' -' and
                                               i[index-2:index] != '+ ' and
                                               i[index-2:index] != ' +')):
                                i = i[:index-1] + i[index:]
                                index = index-1
                            else:
                                break
                        if index > 0 and i[index-1] == ' ': #deletes space before quantity
                            i = i[:index-1]+i[index:]
                        index = i.find(string, index+1)
#                         print(i)
                    else:
                        index = i.find(string, index+1)
        for color in colorsCZ + colorsEN: #filtering colors from products
            index = i.find(color)
            while index != -1:
                if (color != "zelen") or (i[index:index+7] != "zelenin"): 
                    cz = 0
#                     print("REMOVE " + color)
#                     print(i)
                    if color in colorsCZ:
                        cz = 1
                    i = i[:index] + i[index+len(color)+cz:] #for czech terms slices +1 character after (sufix)
                    if (index >= 3) and (i[index-3:index] == " - "): 
                        i = i[:index-3]+i[index:]
                    elif (index > 0) and (i[index-1] == " " or i[index-1] == "-"):
                        i = i[:index-1]+i[index:]
                    index = i.find(color)
                else:
                    index = i.find(color, index+1)
        index = i.find(vel)
        sizes = ["i", "í", "x", "s", "l", "m"] #odstranění označení velikost ...
        while index != -1:
            i = i[:index] + i[index+len(vel):]
            while (len(i)>index) and \
                (i[index] == " " or 
                 i[index] in sizes or
                 (ord(i[index]) >= 48 and ord(i[index]) < 58)):
                i = i[:index]+i[index+1:]
            index = i.find(vel,index+1)
        if len(i)>0 and i[len(i)-1] == " ":
            i = i[:len(i)-1]
        while len(i)>0 and i[0] == " ":
            i = i[1:]
#         print(i)
        setOfNames.loc[setOfNames[name_column]==j, [name_column]] = i #rewrites product name in setOfNames
    
removeQuantityColor(item_content_v_id, name_column)

# material_attributes_cz_no_duplicates.loc[:,"NAME_GROUPING"] = material_attributes_cz_no_duplicates.NAME_GROUPING.str.casefold().str.replace('+ ','').str.replace('+','').str.replace('- ','').str.replace('-','').str.replace('  ',' ') 
# print(material_attributes_cz_no_duplicates)

print('Time to filter: {} mins '.format(round((time() - t) / 60, 2)))

Time to filter: 6.85 mins 


In [99]:
# print(item_content_v_id.iloc[0])
item_content_v_id.loc[:, name_column] = item_content_v_id[name_column].str.replace('\+ ','').str.replace('\+','').str.replace('- ','').str.replace('-','').str.replace('  ',' ') 

In [105]:
#spojední všech částí tabulky
item_content_filtered = item_content_filtered1.append(item_content_filtered2, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered3, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered4, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered5, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered6, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered7, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered8, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered9, ignore_index = True)
print(len(item_content_filtered))
item_content_filtered = item_content_filtered.append(item_content_filtered10, ignore_index = True)
print(len(item_content_filtered))

700000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1294543


In [263]:
#odstranění duplikátů id
# item_content_filtered_no_duplicates = item_content_filtered.drop_duplicates(subset='sid', keep="last")
print(len(item_content_filtered_no_duplicates))
# item_content_filtered_no_duplicates.to_csv(path_or_buf="Data/item_content_filtered_no_duplicates.csv",index=False)
print(item_content_filtered_no_duplicates)

1248406
                  sid                                               name  \
1          1511421004       dámská mikina Logo Hoodie 1895751010 L černá   
2          1511205002                dámské šaty 41.003.82.2525 36 modrá   
3          1018999010                   Zen Pro Wmn Taupe/Coral Red 41,5   
4        100014844258  Plastový kryt - Best Friends pro Samsung Galax...   
5             1390883          Balení 2x ručník a 1x osuška GREEK - Navy   
...               ...                                                ...   
1294538  100004145040        C&K Vocal: Dřív než něco začne (4x CD) - CD   
1294539       1010553                                     Sestava Parana   
1294540    1416452005                  Classic Dusty Green M8W10 (41-42)   
1294541  100003994287  Cantini Barbara: Mortina - Příběh, při kterém ...   
1294542  100020709700  LED stolní lampička 2,5W/3000K/170Lm, černá s ...   

                                             name_grouping  \
1            colu

In [237]:
#groups ID of every product with the same name to list of IDs
item_content_list = item_content_filtered_no_duplicates.groupby('name_grouping')['sid'].apply(list)
print(item_content_list)

name_grouping
                                                                                      [100018360531, 100018360537, 100022733666, 100...
"28"" he cayo al sora 2016 51 zánovní"                                                                                      [917999001]
"challenger 26"" dámské 180 "" zánovní"                                                                                     [932553001]
"dětský tablet řada maxipes fík 7"" použité"                                                                                   [622196]
"čtyři vraždy stačí drahoušku" edice klenoty českého filmu remasterovaná verze dvd                                       [100003591579]
                                                                                                            ...                        
“frau hölle” cappellová tanja: abeceda krasopsaní                                                                        [100003352935]
™ 1120 200 párů                   

In [238]:
#information new dataset of lists of IDs
length = item_content_filtered_no_duplicates.groupby('name_grouping').apply(len)
print(length.sort_values())
print(sum(length))
print(sum(length) - len(length))
print(sum(length == 1))
print(str((sum(length)-len(length))/sum(length)*100) + "% products is the same as other products")

name_grouping
led žárovka zls509 e27 6000k                        1
nines 31421 celia nugolet                           1
nines 31441 celia coleccion mechanická panenka      1
nines 31471 celio barret                            1
nines 31481 celia barret                            1
                                                 ... 
soliver dívčí tričko                              230
seven dívčí tričko                                232
seven chlapecké tričko                            258
topo dívčí šaty                                   362
happy girls dívčí šaty                            504
Length: 838348, dtype: int64
1248406
410058
685341
32.84652588981469% products is the same as other products


In [239]:
print(item_content_list[""]) #najit id produktu ktere byly po sjednocení názvu pojmenovány prázdným řetězcem
print(len(item_content_list[""]))

[100018360531, 100018360537, 100022733666, 100002126251, 100017362505, 100003300853, 100018360532, 100034181664, 757331, 424431]
10


In [240]:
item_list = item_content_list.to_frame() #item_list obsahuje dataframe s produkty, ktere mají více id 
item_list = item_list.reset_index()
item_list.columns = ["name_grouping","list_id"]
print(item_list)

                                            name_grouping  \
0                                                           
1                  "28"" he cayo al sora 2016 51 zánovní"   
2                 "challenger 26"" dámské 180 "" zánovní"   
3            "dětský tablet řada maxipes fík 7"" použité"   
4       "čtyři vraždy stačí drahoušku" edice klenoty č...   
...                                                   ...   
838343  “frau hölle” cappellová tanja: abeceda krasopsaní   
838344                                    ™ 1120 200 párů   
838345                              ™ peltor™ bulls eye i   
838346                               ™ peltor™ optime iii   
838347                                        ™ peltor™ x   

                                                  list_id  
0       [100018360531, 100018360537, 100022733666, 100...  
1                                             [917999001]  
2                                             [932553001]  
3                          

In [242]:
item_list_nonempty = item_list[item_list["name_grouping"] != ""] #nezařazuje produkty které mají po filtrování prázdný název do stejné skupiny
print(len(item_list_withempty))
print(len(item_list_nonempty))

838348
838347


In [243]:
item_list_empty = item_list.loc[item_list["name_grouping"] == "", ["list_id"]] #přidá produkty s prázdným řetězcem všechny do své samotné kategorie
item_list_e = pd.DataFrame(columns = ["name_grouping", "list_id"])
liste = [i for i in list_id_empty.iloc[0]["list_id"]]
for i in liste:
    item_list_e = item_list_e.append({"name_grouping": '',"list_id":[i]},ignore_index = True)
print(item_list_e)
item_list_nonempty = item_list_nonempty.append(item_list_e,ignore_index = True)
print(item_list_nonempty)

   name_grouping         list_id
0                 [100018360531]
1                 [100018360537]
2                 [100022733666]
3                 [100002126251]
4                 [100017362505]
5                 [100003300853]
6                       [503180]
7                       [757330]
8                       [807721]
9                 [100018360532]
10                [100034181664]
11                      [757331]
12                      [424431]
                                            name_grouping         list_id
0                  "28"" he cayo al sora 2016 51 zánovní"     [917999001]
1                 "challenger 26"" dámské 180 "" zánovní"     [932553001]
2            "dětský tablet řada maxipes fík 7"" použité"        [622196]
3       "čtyři vraždy stačí drahoušku" edice klenoty č...  [100003591579]
4       '47 brand kšilt nhl los angeles kings ´47 captain  [100025552567]
...                                                   ...             ...
838355              

In [262]:
print(item_list_nonempty)
item_list_nonempty.to_csv(path_or_buf="Data/item_list.csv",index=False)

                                            name_grouping         list_id
0                  "28"" he cayo al sora 2016 51 zánovní"     [917999001]
1                 "challenger 26"" dámské 180 "" zánovní"     [932553001]
2            "dětský tablet řada maxipes fík 7"" použité"        [622196]
3       "čtyři vraždy stačí drahoušku" edice klenoty č...  [100003591579]
4       '47 brand kšilt nhl los angeles kings ´47 captain  [100025552567]
...                                                   ...             ...
838355                                                           [807721]
838356                                                     [100018360532]
838357                                                     [100034181664]
838358                                                           [757331]
838359                                                           [424431]

[838360 rows x 2 columns]


In [244]:
# product_views_backup = product_views_cz
print(product_views_backup)

          user_id_anon    product_id
0             14723304  100011803451
1             14723304  100011612041
2             14723304  100016624290
3             14723304       1093107
4             14723304  100016287573
...                ...           ...
31502082      54535501     992932006
31502083      54535501       1146359
31502084      54535501  100016023536
31502085      54535501        958947
31502086      54535501        908438

[31502086 rows x 2 columns]


In [245]:
product_views_cz = product_views_backup

In [246]:
item_list = item_list_nonempty
item_list_containing_rewritable = item_list[item_list["list_id"].apply(len)>1] #vyřadí ty listy, ve kterých je pouze jedno id
print(item_list_containing_rewritable)

                                            name_grouping         list_id
0                  "28"" he cayo al sora 2016 51 zánovní"     [917999001]
1                 "challenger 26"" dámské 180 "" zánovní"     [932553001]
2            "dětský tablet řada maxipes fík 7"" použité"        [622196]
3       "čtyři vraždy stačí drahoušku" edice klenoty č...  [100003591579]
4       '47 brand kšilt nhl los angeles kings ´47 captain  [100025552567]
...                                                   ...             ...
838355                                                           [807721]
838356                                                     [100018360532]
838357                                                     [100034181664]
838358                                                           [757331]
838359                                                           [424431]

[838360 rows x 2 columns]
                                 name_grouping  \
33      100 faktů které musíš znát 

In [257]:
pbar2 = ProgressBar()

#funkce přepíše v množině setToRewriteIDs ty ID, které jsou shodné s nějakým jiným ID
def rewriteIDs(setToRewriteIDs, nameColumn, setListIDs, nameListColumn): #upravené pro setListIDs obsahující už množiny s více než jedním prvkem
    for i in pbar2(setListIDs[nameListColumn]):
        setToRewriteIDs.loc[(setToRewriteIDs[nameColumn].isin(i[1:])), nameColumn] = i[0]        

t2 = time()

prod_views = product_views_cz.iloc[17000000:]
print(prod_views)
rewriteIDs(prod_views, "product_id", item_list_containing_rewritable, "list_id")
print(prod_views)
print('Time to filter: {} mins '.format(round((time() - t2) / 60, 2)))
    

          user_id_anon    product_id
17000001      27059382    1404259001
17000002      27059382       2491241
17000003      27059382       1194284
17000004      27059382  100022208562
17000005      27059382       1491873
...                ...           ...
31502082      54535501     992932006
31502083      54535501       1146359
31502084      54535501  100016023536
31502085      54535501        958947
31502086      54535501        908438

[14502086 rows x 2 columns]


c:\users\pajav\source\repos\lecture4\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
100% |########################################################################|


          user_id_anon    product_id
17000001      27059382    1404259002
17000002      27059382       2491241
17000003      27059382       1194284
17000004      27059382  100022208562
17000005      27059382       1491873
...                ...           ...
31502082      54535501     992932002
31502083      54535501       1146359
31502084      54535501  100016023536
31502085      54535501        958947
31502086      54535501        908438

[14502086 rows x 2 columns]
Time to filter: 710.4 mins 


In [259]:
product_viewes_unified = prod_views1 #spojení jednotlivých tabulek pro product_views
product_viewes_unified = product_viewes_unified.append(prod_views2, ignore_index = True)
product_viewes_unified = product_viewes_unified.append(prod_views3, ignore_index = True)
print(product_viewes_unified)

          user_id_anon    product_id
0             14723304  100011803451
1             14723304  100011612041
2             14723304  100016624290
3             14723304       1093107
4             14723304  100016287573
...                ...           ...
31502081      54535501     992932002
31502082      54535501       1146359
31502083      54535501  100016023536
31502084      54535501        958947
31502085      54535501        908438

[31502086 rows x 2 columns]


In [260]:
product_viewes_unified.to_csv(path_or_buf="Data/product_views_unified.csv",index=False)